In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from operator import itemgetter

c:\users\tonyl\appdata\local\programs\python\python38\lib\site-packages\setuptools\distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


In [2]:
train = pd.read_csv('./data/train.csv')

In [3]:
train.groupby(['Sex 0M1F','label']).mean()

id  MO HLADR+ MFI (cells/ul)  Neu CD64+MFI (cells/ul)  \
Sex 0M1F label                                                                 
0        0      51.387097               1354.000000                99.000000   
         1      37.714286               1109.642857               597.571429   
1        0      33.703704               1485.111111               110.555556   
         1      47.333333                831.466667               710.133333   

                CD3+T (cells/ul)  CD8+T (cells/ul)  CD4+T (cells/ul)  \
Sex 0M1F label                                                         
0        0           1004.276452        483.150968        502.179677   
         1            610.707143        269.370000        365.400000   
1        0           1350.175185        670.417407        667.587037   
         1            623.093333        323.502000        289.908667   

                NK (cells/ul)  CD19+ (cells/ul)  CD45+ (cells/ul)        Age  \
Sex 0M1F label                                                                 
0        0         293.086129        122.593871       1434.754839  42.451613   
         1         100.105714         64.545000        782.853571  41.142857   
1        0         242.642222        168.151852       1771.611481  37.000000   
         1          97.951333         72.653333        800.835333  40.533333   

                Mono CD64+MFI (cells/ul)  
Sex 0M1F label                            
0        0                   1701.000000  
         1                   2880.000000  
1        0                   1790.000000  
         1                   2536.133333

In [207]:
train.dropna(inplace=True)
x = train.drop(['id', 'label', 'Age'], axis=1)
y = train['label']

X_train, X_val, y_train, y_val = train_test_split(x, y, test_size=0.1)

In [208]:
class_weights = [{0:1, 1:1},{0:1, 1:2},{0:1, 1:3}]
penal = ['l2', 'none']
best = 0
for p in penal:
    for c in class_weights:
        logis = LogisticRegression(penalty=p, class_weight=c, solver='lbfgs', max_iter=1000)
        logis.fit(X_train, y_train)

        y_pred = logis.predict(X_val)
        acc = accuracy_score(y_val, y_pred)
        if best < acc:
            best = acc
            best_comb = (p,c,logis)
print(best, best_comb)

0.8888888888888888 ('l2', {0: 1, 1: 1}, LogisticRegression(class_weight={0: 1, 1: 1}, max_iter=1000))


In [209]:
n = 20
est = range(20,40,1)
feat = range(4,7)
depth = range(2,7)
accuracy = {}
for epoch in range(n):
    for d in depth:
        for i in est:
            for j in feat:
                rf_model = RandomForestClassifier(max_features=j, n_estimators=i, max_depth=d)
                rf_model.fit(X_train, y_train)

                y_pred = rf_model.predict(X_val)
                acc = accuracy_score(y_val, y_pred)
                if (i,j,d) in accuracy:
                    accuracy[(i,j,d)] = accuracy[(i,j,d)] + acc/n
                else:
                    accuracy[(i,j,d)] = acc/n
print(dict(sorted(accuracy.items(), key = itemgetter(1), reverse = True)[:5]))

{(29, 4, 8): 0.8388888888888886, (20, 5, 6): 0.8277777777777775, (23, 5, 7): 0.822222222222222, (23, 5, 8): 0.8222222222222219, (35, 6, 8): 0.8222222222222219}


In [210]:
rf_model = RandomForestClassifier(max_features=4, n_estimators=23, max_depth=6, random_state=80)
rf_model.fit(x, y)

RandomForestClassifier(max_depth=6, max_features=4, n_estimators=23,
                       random_state=80)

In [177]:
test = pd.read_csv('./data/test.csv')
X_test = test.drop(['id', 'Age'], axis=1)
y_pred = rf_model.predict(X_test)

In [178]:
submission = test.assign(label=y_pred)
submission = submission[['id', 'label']]
submission.to_csv('./data/submission.csv', index=False)